In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_name = "NF-CSE-CIC-IDS2018-v2.parquet"
data = pd.read_parquet(file_name)

In [3]:
##ADDED
file_name_test = "NF-CSE-CIC-IDS2018-v3.parquet"
test_data = pd.read_parquet(file_name_test)
##

In [4]:
data.Label.value_counts()

Label
0    16635567
1     2258141
Name: count, dtype: int64

In [5]:
##ADDED
test_data.Label.value_counts()
##

Label
0    17514626
1     2600903
Name: count, dtype: int64

In [6]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [7]:
##ADDED
test_data.rename(columns=lambda x: x.strip(), inplace=True)
test_data['IPV4_SRC_ADDR'] = test_data["IPV4_SRC_ADDR"].apply(str)
test_data['L4_SRC_PORT'] = test_data["L4_SRC_PORT"].apply(str)
test_data['IPV4_DST_ADDR'] = test_data["IPV4_DST_ADDR"].apply(str)
test_data['L4_DST_PORT'] = test_data["L4_DST_PORT"].apply(str)
##

In [8]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [9]:
##ADDED
test_data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)
##

In [10]:
data.Attack.unique()

array(['SSH-Bruteforce', 'Benign', 'DDoS attacks-LOIC-HTTP',
       'DDOS attack-HOIC', 'DoS attacks-Slowloris', 'DoS attacks-Hulk',
       'FTP-BruteForce', 'Infilteration', 'Bot', 'DoS attacks-GoldenEye',
       'Brute Force -Web', 'DoS attacks-SlowHTTPTest', 'SQL Injection',
       'DDOS attack-LOIC-UDP', 'Brute Force -XSS'], dtype=object)

In [11]:
##ADDED
test_data.Attack.unique()
##

array(['Benign', 'FTP-BruteForce', 'SSH-Bruteforce',
       'DoS_attacks-GoldenEye', 'DoS_attacks-Slowloris',
       'DoS_attacks-SlowHTTPTest', 'DoS_attacks-Hulk',
       'DDoS_attacks-LOIC-HTTP', 'DDOS_attack-LOIC-UDP',
       'DDOS_attack-HOIC', 'Brute_Force_-Web', 'Brute_Force_-XSS',
       'SQL_Injection', 'Infilteration', 'Bot'], dtype=object)

In [12]:
##ADDED
data["Attack"] = data["Attack"].str.replace(" ", "_")
##

In [13]:
data = data.groupby(by='Attack').sample(frac=0.02, random_state=13)

In [14]:
##ADDED
test_data = test_data.groupby(by='Attack').sample(frac=0.02, random_state=13)
##

In [15]:
data.groupby(by="Attack").count()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,332711,332711,332711,332711,332711,332711,332711,332711,332711,332711,...,332711,332711,332711,332711,332711,332711,332711,332711,332711,332711
Bot,2862,2862,2862,2862,2862,2862,2862,2862,2862,2862,...,2862,2862,2862,2862,2862,2862,2862,2862,2862,2862
Brute_Force_-Web,43,43,43,43,43,43,43,43,43,43,...,43,43,43,43,43,43,43,43,43,43
Brute_Force_-XSS,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
DDOS_attack-HOIC,21617,21617,21617,21617,21617,21617,21617,21617,21617,21617,...,21617,21617,21617,21617,21617,21617,21617,21617,21617,21617
DDOS_attack-LOIC-UDP,42,42,42,42,42,42,42,42,42,42,...,42,42,42,42,42,42,42,42,42,42
DDoS_attacks-LOIC-HTTP,6146,6146,6146,6146,6146,6146,6146,6146,6146,6146,...,6146,6146,6146,6146,6146,6146,6146,6146,6146,6146
DoS_attacks-GoldenEye,554,554,554,554,554,554,554,554,554,554,...,554,554,554,554,554,554,554,554,554,554
DoS_attacks-Hulk,8653,8653,8653,8653,8653,8653,8653,8653,8653,8653,...,8653,8653,8653,8653,8653,8653,8653,8653,8653,8653


In [16]:
##ADDED
test_data.groupby(by="Attack").count()
##

,FLOW_START_MILLISECONDS,FLOW_END_MILLISECONDS,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,FTP_COMMAND_RET_CODE,SRC_TO_DST_IAT_MIN,SRC_TO_DST_IAT_MAX,SRC_TO_DST_IAT_AVG,SRC_TO_DST_IAT_STDDEV,DST_TO_SRC_IAT_MIN,DST_TO_SRC_IAT_MAX,DST_TO_SRC_IAT_AVG,DST_TO_SRC_IAT_STDDEV,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,350293,350293,350293,350293,350293,350293,350293,350293,350293,350293,...,350293,350293,350293,350293,350293,350293,350293,350293,350293,350293
Bot,4154,4154,4154,4154,4154,4154,4154,4154,4154,4154,...,4154,4154,4154,4154,4154,4154,4154,4154,4154,4154
Brute_Force_-Web,32,32,32,32,32,32,32,32,32,32,...,32,32,32,32,32,32,32,32,32,32
Brute_Force_-XSS,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
DDOS_attack-HOIC,20646,20646,20646,20646,20646,20646,20646,20646,20646,20646,...,20646,20646,20646,20646,20646,20646,20646,20646,20646,20646
DDOS_attack-LOIC-UDP,69,69,69,69,69,69,69,69,69,69,...,69,69,69,69,69,69,69,69,69,69
DDoS_attacks-LOIC-HTTP,5772,5772,5772,5772,5772,5772,5772,5772,5772,5772,...,5772,5772,5772,5772,5772,5772,5772,5772,5772,5772
DoS_attacks-GoldenEye,1226,1226,1226,1226,1226,1226,1226,1226,1226,1226,...,1226,1226,1226,1226,1226,1226,1226,1226,1226,1226
DoS_attacks-Hulk,2002,2002,2002,2002,2002,2002,2002,2002,2002,2002,...,2002,2002,2002,2002,2002,2002,2002,2002,2002,2002


In [17]:
# X = data.drop(columns=["Attack", "Label"])
# y = data[["Attack", "Label"]]

# X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.3, random_state=13, stratify=y)
# ADDED
X = test_data.drop(columns=["Attack", "Label", "FLOW_START_MILLISECONDS", "FLOW_END_MILLISECONDS",
                                 "SRC_TO_DST_IAT_MIN", "SRC_TO_DST_IAT_MAX", "SRC_TO_DST_IAT_AVG",
                                 "SRC_TO_DST_IAT_STDDEV", "DST_TO_SRC_IAT_MIN", "DST_TO_SRC_IAT_MAX",
                                 "DST_TO_SRC_IAT_AVG", "DST_TO_SRC_IAT_STDDEV"])
y = test_data[["Attack", "Label"]]
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)
##

In [18]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [19]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [20]:
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [21]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h
18744204,172.31.69.14,18.219.211.138,4.740102e-07,1.323114e-06,0.001264,0.000012,0.000801,0.000012,1.045428e-06,1.016509e-06,...,0.0,0.019249,0.063169,3.165538e-07,3.165538e-07,4.626324e-07,4.626324e-07,0.000000,3.038169e-07,"[4.7401022514464873e-07, 1.3231141133088036e-0..."
16092777,172.31.64.33,169.254.169.254,8.691857e-08,2.426175e-07,0.000155,0.000002,0.000196,0.000002,1.177649e-07,1.037055e-07,...,0.0,0.003530,0.007722,5.804601e-08,5.804601e-08,8.483223e-08,8.483223e-08,0.000000,5.571047e-08,"[8.691857374655049e-08, 2.426175334037123e-07,..."
8585764,18.219.9.1,172.31.69.28,3.946205e-07,1.101512e-06,0.001047,0.000010,0.002244,0.000010,8.703343e-07,8.462588e-07,...,0.0,0.128201,0.052589,2.635357e-07,2.635357e-07,3.851483e-07,3.851483e-07,0.000000,2.529321e-07,"[3.946205413164243e-07, 1.1015121191910754e-06..."
9151507,18.218.229.235,172.31.69.28,2.759324e-07,7.702155e-07,0.000756,0.000007,0.001569,0.000007,6.085679e-07,5.917335e-07,...,0.0,0.089643,0.036772,1.842733e-07,1.842733e-07,2.693090e-07,2.693090e-07,0.000000,1.768589e-07,"[2.759323680481234e-07, 7.702154745117166e-07,..."
14212479,172.31.65.38,172.31.0.2,8.558578e-07,8.375243e-07,0.003386,0.000059,0.010988,0.000059,8.503230e-07,8.503230e-07,...,0.0,0.000000,0.000000,8.001776e-06,8.001776e-06,6.389747e-06,8.525785e-07,0.003564,7.679815e-06,"[8.558577987023315e-07, 8.375242601934382e-07,..."


In [22]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [23]:
# Training graph

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [24]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [25]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [26]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [27]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [28]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [29]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)
  
dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [30]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [31]:
# Convert to GPU
train_g = train_g.to('cuda')

In [32]:
dgi.load_state_dict(torch.load('best_dgi_CSE_v2.pkl'))

/tmp/ipykernel_54986/1688994637.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dgi.load_state_dict(torch.load('best_dgi_CSE_v2.pkl'))


<All keys matched successfully>

In [33]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [34]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [35]:
# Convert to GPU
test_g = test_g.to('cuda')

In [36]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [37]:
df_train = pd.DataFrame(training_emb, )
df_train["Attack"] = lab_enc.inverse_transform(
        train_g.edata['Attack'].detach().cpu().numpy())
df_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb, )
df_test["Attack"] = lab_enc.inverse_transform(
        test_g.edata['Attack'].detach().cpu().numpy())
df_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

In [38]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,Attack,Label
0,0.046952,-0.010453,0.016254,0.037051,-0.003613,-0.028452,-0.014636,0.064069,0.010517,0.046255,...,-0.004440,-0.041173,-0.041851,0.021108,0.020684,0.009840,-0.024127,-0.019639,Bot,1
1,0.046952,-0.010453,0.016254,0.037051,-0.003613,-0.028452,-0.014636,0.064069,0.010517,0.046255,...,-0.004440,-0.041173,-0.041851,0.021108,0.020684,0.009840,-0.024127,-0.019639,Bot,1
2,0.046952,-0.010453,0.016254,0.037051,-0.003613,-0.028452,-0.014636,0.064069,0.010517,0.046255,...,-0.004440,-0.041173,-0.041851,0.021108,0.020684,0.009840,-0.024127,-0.019639,Bot,1
3,0.046952,-0.010453,0.016254,0.037051,-0.003613,-0.028452,-0.014636,0.064069,0.010517,0.046255,...,-0.004440,-0.041173,-0.041851,0.021108,0.020684,0.009840,-0.024127,-0.019639,Bot,1
4,0.046952,-0.010453,0.016254,0.037051,-0.003613,-0.028452,-0.014636,0.064069,0.010517,0.046255,...,-0.004440,-0.041173,-0.041851,0.021108,0.020684,0.009840,-0.024127,-0.019639,Bot,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561750,0.057992,0.003622,0.012290,0.025195,0.002873,-0.043223,-0.007850,0.071469,0.020922,0.065263,...,-0.005155,-0.043627,-0.042249,0.027284,0.012259,0.022695,-0.027635,-0.002147,Benign,0
561751,0.066057,0.011006,0.009527,-0.019138,0.018792,-0.085744,-0.021677,0.044442,0.060225,0.069562,...,-0.023375,-0.049153,-0.011907,0.006916,-0.014432,0.017769,-0.028692,0.013128,Benign,0
561752,0.051596,-0.003059,0.023261,0.026722,0.002785,-0.036216,-0.002702,0.055322,0.017842,0.051991,...,-0.008060,-0.060453,-0.038828,0.031201,0.003527,0.013831,-0.022317,-0.007939,Benign,0
561753,0.108938,0.001880,-0.028985,-0.120018,0.034612,-0.192602,0.052352,0.067792,0.101796,0.134094,...,0.039135,0.008710,0.003210,-0.071038,-0.019164,-0.004610,0.013132,-0.009407,Benign,0


# Embeddings CBLOF  Embeddings

In [39]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [40]:
benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label", "Attack"])
normal_train_samples = df_train.drop(columns=["Label", "Attack"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label", "Attack"])

In [41]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [43]:
from pyod.models.cblof import CBLOF
n_est = [2,3,5,7,9,10]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(benign_train_samples)
    except:
        continue
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [01:33<00:00,  2.60s/it]

{'n_estimators': 9, 'con': 0.2}
0.5784377547849563
              precision    recall  f1-score   support

           0     0.9039    0.7985    0.8479    209540
           1     0.2411    0.4299    0.3090     31212

    accuracy                         0.7507    240752
   macro avg     0.5725    0.6142    0.5784    240752
weighted avg     0.8180    0.7507    0.7780    240752



In [45]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(normal_train_samples)
    except:
        continue
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [01:50<00:00,  3.06s/it]

{'n_estimators': 9, 'con': 0.2}
0.4989703809579025
              precision    recall  f1-score   support

           0     0.8715    0.8022    0.8354    209540
           1     0.1342    0.2059    0.1625     31212

    accuracy                         0.7249    240752
   macro avg     0.5029    0.5040    0.4990    240752
weighted avg     0.7759    0.7249    0.7482    240752



In [51]:
# HBOS  Embeddings

In [52]:
benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label", "Attack"])
normal_train_samples = df_train.drop(columns=["Label", "Attack"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label", "Attack"])

In [53]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [54]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [06:17<00:00, 10.48s/it]

{'n_estimators': 30, 'con': 0.001}
0.4650038777174577
              precision    recall  f1-score   support

           0     0.8702    0.9986    0.9300    209540
           1     0.0000    0.0000    0.0000     31212

    accuracy                         0.8692    240752
   macro avg     0.4351    0.4993    0.4650    240752
weighted avg     0.7574    0.8692    0.8094    240752



In [55]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [05:27<00:00,  9.10s/it]

{'n_estimators': 5, 'con': 0.001}
0.4649099411461389
              precision    recall  f1-score   support

           0     0.8702    0.9983    0.9298    209540
           1     0.0000    0.0000    0.0000     31212

    accuracy                         0.8688    240752
   macro avg     0.4351    0.4991    0.4649    240752
weighted avg     0.7573    0.8688    0.8093    240752



In [59]:
##  PCA  Emb

In [60]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [02:18<00:00,  3.84s/it]

{'n_estimators': 10, 'con': 0.001}
0.4649693985039269
              precision    recall  f1-score   support

           0     0.8702    0.9985    0.9299    209540
           1     0.0000    0.0000    0.0000     31212

    accuracy                         0.8691    240752
   macro avg     0.4351    0.4993    0.4650    240752
weighted avg     0.7574    0.8691    0.8094    240752



In [61]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [03:29<00:00,  5.82s/it]

{'n_estimators': 10, 'con': 0.001}
0.46489448006845735
              precision    recall  f1-score   support

           0     0.8702    0.9982    0.9298    209540
           1     0.0000    0.0000    0.0000     31212

    accuracy                         0.8688    240752
   macro avg     0.4351    0.4991    0.4649    240752
weighted avg     0.7573    0.8688    0.8092    240752



In [65]:
##  IF  Emb

In [66]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

  0%|          | 0/24 [00:00<?, ?it/s]

100%|██████████| 24/24 [01:32<00:00,  3.84s/it]

{'n_estimators': 20, 'con': 0.001}
0.46534689542907653
              precision    recall  f1-score   support

           0     0.8703    0.9991    0.9302    209540
           1     0.0355    0.0002    0.0004     31212

    accuracy                         0.8696    240752
   macro avg     0.4529    0.4997    0.4653    240752
weighted avg     0.7621    0.8696    0.8097    240752



In [67]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [01:20<00:00,  3.36s/it]

{'n_estimators': 20, 'con': 0.001}
0.46538420110407863
              precision    recall  f1-score   support

           0     0.8702    0.9985    0.9299    209540
           1     0.0387    0.0004    0.0008     31212

    accuracy                         0.8691    240752
   macro avg     0.4545    0.4994    0.4654    240752
weighted avg     0.7624    0.8691    0.8095    240752

